# MIT License

Copyright 2020 Pablo Duboue

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

This code is derived from the Case Study in Chapter 10, see https://github.com/DrDub/artfeateng

# Geographical Features

In GIS features, we would look at using representions based on distance to key points. 

## Example Data

We will be using the migration data through satellite telemetry for African cuckoos in Nigeria, kindly released by [Iwajomo SB et al, 2018 as part of the movebank.org data repository](https://www.datarepository.movebank.org/handle/10255/move.714). It contains 12,563 tracking points for six individuals, from May 29th, 2013 until Jun 28th, 2017.

![Data trajectory](../media/movebank.png)

We will look into the problem of predicting whether a bird will move in the next two hours.

In [1]:
# from CELL 21
import os

rows = list()
with open(os.path.join("..", "data", "African cuckoo in Nigeria (data from Iwajomo et al. 2018).csv")) as csv:
    header = None
    for row in csv:
        cols = row.split(",")
        if header is None:
            header = { w: i for i, w in enumerate(cols) }
        else:
            long =  cols[header['location-long']]
            lat = cols[header['location-lat']]
            if len(long) > 3 and len(lat) > 3:
                rows.append( [ 
                    cols[header['timestamp']],
                    long,
                    lat,
                    cols[header['individual-local-identifier']]
                ])
print("Read", len(rows), "rows")

Read 12096 rows


We will now split the rows by individual, leaving two individuals for training, two for devtest and one for final testing.

To obtain the classes (moved or not), we parse the time stamps and assign a class depending on the number of seconds between two entries. We will also explode the date and time as individual features and compute a "day in the year" feature.

In [2]:
# from CELL 22
import random

individuals = set(map(lambda x:x[-1], rows))

print("Total individuals:", len(individuals))

for indiv in individuals:
    print(indiv, len(list(filter(lambda x:x[-1] == indiv, rows))))

random_indiv = list(individuals)
random.shuffle(random_indiv, random=random.Random(40).random)
train_indiv = set(random_indiv[0:2])
test_indiv = set(random_indiv[2:4])
final_test_indiv = set(random_indiv[4])

train_rows = len(list(filter(lambda x:x[-1] in train_indiv, rows)))
print("Total train rows:", train_rows)
test_rows = len(list(filter(lambda x:x[-1] in test_indiv, rows)))
print("Total test rows:", test_rows)

Total individuals: 5
"126698" 712
"126695" 136
"126697" 3063
"126696" 3617
"126694" 4568
Total train rows: 4704
Total test rows: 4329


In [3]:
# from CELL 23
import re
import datetime
import numpy as np

xtrain = np.zeros((train_rows, 
                   3 + # year, month, day 
                   1 + # day of year
                   3 + # hh, mm, ss
                   2 # lat long
                  ), dtype='float32')
ytrain = np.zeros((train_rows,), dtype='float32')

prev = None
for idx, row in enumerate(filter(lambda x:x[-1] in train_indiv, rows)):
    year, month, day, hh, mm, ss = re.split(" |-|:+", row[0])
    year = int(year)
    month = int(month)
    day = int(day)
    hh = int(hh)
    mm = int(hh)
    ss = float(ss)
    dt = datetime.datetime(year, month, day, hh, mm, int(ss))
    try:
        xtrain[idx,:] = [ 
            year, month, day, dt.timetuple().tm_yday, hh, mm, ss,
            float(row[1]), float(row[2])
        ]
    except:
        print(row)
    ts = dt.timestamp()
    if prev is None:
        ytrain[idx] = 0.0
    else:
        if ts - prev > 2 * 60 * 60:
            ytrain[idx] = 1.0
        else:
            ytrain[idx] = 0.0
    prev = ts 
print(np.sum(ytrain))

xtest = np.zeros((test_rows, xtrain.shape[1]), dtype='float32')
ytest = np.zeros((test_rows,), dtype='float32')

prev = None
for idx, row in enumerate(filter(lambda x:x[-1] in test_indiv, rows)):
    year, month, day, hh, mm, ss = re.split(" |-|:+", row[0])
    year = int(year)
    month = int(month)
    day = int(day)
    hh = int(hh)
    mm = int(hh)
    ss = float(ss)
    dt = datetime.datetime(year, month, day, hh, mm, int(ss))
    try:
        xtest[idx,:] = [ 
            year, month, day, dt.timetuple().tm_yday, hh, mm, ss,
            float(row[1]), float(row[2])
        ]
    except:
        print(row)
    ts = dt.timestamp()
    if prev is None:
        ytest[idx] = 0.0
    else:
        if ts - prev > 2 * 60 * 60:
            ytest[idx] = 1.0
        else:
            ytest[idx] = 0.0
    prev = ts 

print(np.sum(ytest))

1727.0
1571.0


In [4]:
# from CELL 24
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support

rf = RandomForestClassifier(n_estimators = 100, random_state=42)

print("Training on {:,} instances for {:,} features".format(*xtrain.shape))
rf.fit(xtrain, ytrain)
ytest_pred = rf.predict(xtest)

prec, rec, f1, _ = precision_recall_fscore_support(ytest, ytest_pred)

print("[Base] Prec ", prec, "Rec", rec, "F1", f1)

/home/pablo/local/virtualenv/feateng/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


Training on 4,704 instances for 9 features
[Base] Prec  [0.73836276 0.62273476] Rec [0.83393764 0.48122215] F1 [0.78324536 0.54290844]


## Radial distance

We will use a list of administrative units obtained from [GeoNames.org](http://download.geonames.org/export/dump/) for both Nigeria and Cameroon and add features as the distance to each of these points.

In [5]:
# from CELL 25
import geopy.distance

adms = list() # of coordinate tuples

with open(os.path.join("..", "data", "NG.txt")) as cm:
    for line in cm:
        fields = line.split("\t")
        if fields[7] == 'ADM2':
            adms.append( (fields[1], (float(fields[4]), float(fields[5]))) )
with open(os.path.join("..", "data", "CM.txt")) as cm:
    for line in cm:
        fields = line.split("\t")
        if fields[7] == 'ADM2':
            adms.append( (fields[1], (float(fields[4]), float(fields[5]))) )

# center for points
lat_center, lon_center = xtrain[:,-2].mean(), xtrain[:,-1].mean()
            
print("Total cities:", len(adms))

close_cities = list()
for adm_coord in adms:
    dist = geopy.distance.distance( (lat_center, lon_center), adm_coord[1])
    if dist.km < 50:
        close_cities.append(adm_coord[1])
        print(adm_coord[0], dist)
print("Close cities:", len(close_cities))

Total cities: 850
Sumaila 35.267436061479415 km
Tudun Wada 32.059023926201526 km
Doguwa 21.28593558849602 km
Ningi 48.061818326436985 km
Kuban 45.62228782290097 km
Close cities: 5


In [6]:
# from CELL 25
import geopy.distance

xtrain_radial = np.zeros((xtrain.shape[0], xtrain.shape[1] + len(close_cities)), dtype='float32')
xtrain_radial[:,:xtrain.shape[1]] = xtrain

for row_idx in range(xtrain.shape[0]):
    for idx, adm_coord in enumerate(close_cities):
        xtrain_radial[row_idx][xtrain.shape[1] + idx] = \
          geopy.distance.distance((xtrain[row_idx][-2], xtrain[row_idx][-1]), adm_coord).km

xtest_radial = np.zeros((xtest.shape[0], xtest.shape[1] + len(close_cities)), dtype='float32')
xtest_radial[:,:xtest.shape[1]] = xtest

for row_idx in range(xtest.shape[0]):
    for idx, adm_coord in enumerate(close_cities):
        xtest_radial[row_idx][xtest.shape[1] + idx] = \
          geopy.distance.distance((xtest[row_idx][-2], xtest[row_idx][-1]), adm_coord).km

print("Training on {:,} instances for {:,} features".format(*xtrain_radial.shape))
rf = RandomForestClassifier(n_estimators=100, random_state=42)

rf.fit(xtrain_radial, ytrain)
ytest_pred_radial = rf.predict(xtest_radial)

prec, rec, f1, _ = precision_recall_fscore_support(ytest, ytest_pred_radial)

print("[Radial] Prec ", prec, "Rec", rec, "F1", f1)

Training on 4,704 instances for 14 features
[Radial] Prec  [0.7160615  0.55267296] Rec [0.79369108 0.44748568] F1 [0.75288048 0.49454801]
